In [12]:
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter

# Parameters
num_requests = 100
arrival_interval = (1, 3)  # minutes
service_time_min = 2  # minutes
service_time_max = 5  # minutes

# Initialize data structures
arrival_times = []
service_start_times = []
service_end_times = []

# Generate random arrival times
current_time = 0
for i in range(num_requests):
    arrival_time = current_time + random.randint(*arrival_interval)
    arrival_times.append(arrival_time)
    current_time = arrival_time

# Generate random service times
service_times = [random.randint(service_time_min, service_time_max) for _ in range(num_requests)]

# Simulation of queueing and service
server_finish_time = [0, 0]  # Finish times for server 1 and server 2
waiting_times = []
service_start_times = []
service_end_times = []

# Initialize variables for calculating delays
delay_per_request = []
total_time_to_finish_all = 0

# Initialize the figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Initialize empty lines for plotting
line_arrival, = ax.step([], [], label='Arrival Time', where='post', color='b')
line_service_start, = ax.step([], [], label='Service Start Time', where='post', color='g')
line_service_end, = ax.step([], [], label='Service End Time', where='post', color='r')

# Function to update the plot for each frame of the animation
def update(frame):
    ax.set_xlim(0, max(max(arrival_times), max(service_end_times)) + 10)
    ax.set_ylim(-1, num_requests + 1)
    ax.set_xlabel('Time (minutes)')
    ax.set_ylabel('Request Number')
    ax.set_title(f'Queueing System Simulation - Time: {frame} seconds')

    # Plot arrival times
    line_arrival.set_data(arrival_times[:frame+1], range(frame+1))
    # Plot service start times
    line_service_start.set_data(service_start_times[:frame+1], range(frame+1))
    # Plot service end times
    line_service_end.set_data(service_end_times[:frame+1], range(frame+1))

    ax.legend()

    # Calculate statistics
    if frame > 0:
        avg_waiting_time = sum(waiting_times[:frame]) / frame
        avg_service_time = sum(service_times) / num_requests
        avg_delay_per_request = sum(delay_per_request) / len(delay_per_request) if delay_per_request else 0
        ax.text(0.02, 0.95, f'Avg Waiting Time: {avg_waiting_time:.2f} minutes', transform=ax.transAxes, fontsize=10, verticalalignment='top')
        ax.text(0.02, 0.90, f'Avg Service Time: {avg_service_time:.2f} minutes', transform=ax.transAxes, fontsize=10, verticalalignment='top')
        ax.text(0.02, 0.85, f'Avg Delay per Request: {avg_delay_per_request:.2f} minutes', transform=ax.transAxes, fontsize=10, verticalalignment='top')

# Simulation loop
for i in range(num_requests):
    # Determine which server the request will use (the one that will be free the earliest)
    next_server = 0 if server_finish_time[0] <= server_finish_time[1] else 1
    
    # Calculate waiting time
    waiting_time = max(0, server_finish_time[next_server] - arrival_times[i])
    waiting_times.append(waiting_time)
    
    # Service starts after waiting time
    service_start_time = arrival_times[i] + waiting_time
    service_start_times.append(service_start_time)
    
    # Service ends
    service_end_time = service_start_time + service_times[i]
    service_end_times.append(service_end_time)
    
    # Update server finish time
    server_finish_time[next_server] = service_end_time
    
    # Calculate delay per request
    delay = waiting_time
    delay_per_request.append(delay)
    
    # Calculate total time to finish all requests
    total_time_to_finish_all = max(total_time_to_finish_all, service_end_time)

# Animate the plot
ani = FuncAnimation(fig, update, frames=max(service_end_times) + 10, interval=1000)
plt.close()  # Close the initial plot window to prevent duplicate display

# Save the animation as GIF file
writer = PillowWriter(fps=10)
ani.save('queueing_animation.gif', writer=writer)

print("Animation saved as queueing_animation.gif")
print(f'Avg Delay per Request: {sum(delay_per_request) / len(delay_per_request):.2f} minutes')
print(f'Total Time to Finish All Requests: {total_time_to_finish_all} minutes')


ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (100,) and arg 1 with shape (101,).